# split dataframe by key and do stuff over partitions

allows joining / grouping

 1. map column(s) to hash, mod to map rows to partition
 2. create individual file for each partition
 3. if several dfs have been partitioned, can do joins on partitions separately (and in parallel)

In [17]:
def make_test_data(n, filename=None):
    chars = [chr(i) for i in range(ord('a'), ord('z') + 1)]
    df = pd.DataFrame({"key": np.random.choice(chars, size=n),
                       "val": np.random.rand(n)})
    if filename is not None:
        df.to_csv(filename, index=False)
    return df

In [118]:
# test join
DATA_DIR = "../data"
names = [os.path.join(DATA_DIR, "test%d.csv" % i) for i in [0, 1]]
dirnames = [n + ".parts" for n in names]

df = [make_test_data(401, n) for n in names]

In [119]:
for name, dirname in zip(names, dirnames):
    init_dir(dirname)

    for _df in pd.read_csv(os.path.join(path, name), chunksize=50):
        to_csv(_df, "key", 7, dirname)

In [124]:
dirname = "../data/join.parts"

init_dir(dirname)
for _df1, _df2 in zip(*(iter_parts(d) for d in dirnames)):
    to_csv(_df1.merge(_df2, on="key"), "key", 7, dirname)

In [127]:
expected = (df[0]
            .merge(df[1], on="key")
            .sort_values(["key", "val_x", "val_y"])
            .reset_index(drop=True))
actual = (pd
          .concat(iter_parts(dirname), axis=0)
          .sort_values(["key", "val_x", "val_y"])
          .reset_index(drop=True))
assert np.all(expected["key"] == actual["key"]) & np.allclose(expected.iloc[:, 1:], actual.iloc[:, 1:])